# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%reload_ext dotenv
%dotenv

import dask
import dask.dataframe as dd

# Configure Dask
dask.config.set({'dataframe.query-planning': True})



c:\Users\malku\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(
C:\Users\malku\AppData\Local\Temp\ipykernel_10480\1330877845.py:6: DeprecationWarning: The current Dask DataFrame implementation is deprecated. 
In a future release, Dask DataFrame will use new implementation that
contains several improvements including a logical query planning.
The user-facing DataFrame API will remain unchanged.

The new implementation is already available and can be enabled by
installing the dask-expr library:

    $ pip install dask-expr

and turning the query planning option on:

    >>> import dask
    >>> dask.config.set({'dataframe.query-planning': True})
    >>> import dask.dataframe as dd

API documentation for the new implementation is available at
https://docs.dask.org/en/stable/dask-expr-api.html

Any feedback can b

In [72]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [75]:
import os
from glob import glob
price_data_dir = os.getenv('PRICE_DATA')
parquet_files = glob(os.path.join(price_data_dir, '**/*.parquet'), recursive=True)
print(price_data_dir)
print(f"Found {len(parquet_files)} parquet files in {price_data_dir}")

parquet_files

../../05_src/data/prices/
Found 0 parquet files in ../../05_src/data/prices/


[]

In [37]:
import os
from glob import glob

# Write your code below.
# Get the PRICE_DATA path
price_data_dir = os.getenv('PRICE_DATA')

# Find all parquet files
parquet_files = glob(os.path.join(price_data_dir, '*/*.parquet'))
dd_px = dd.read_parquet(parquet_files).set_index('ticker')

# Logging for debugging
# print(f"Found {len(parquet_files)} parquet files in {price_data_dir}")



IsADirectoryError: Path c:/Yuliia/DSI/scaling_to_production/05_src/data/prices/A/A_2000.parquet points to a directory, but only file paths are supported. To construct a nested or union dataset pass a list of dataset objects instead.

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [33]:
import numpy as np
dd_px = dd.read_parquet(parquet_files).set_index('ticker')
dd_feat = (dd_px.groupby('ticker', group_keys=False). apply(
                          lambda x: x.assign (Close_lag = x['Close'].shift (1))
). assign(
    returns = lambda x: x['Close']/x[ 'Close_lag'] - 1
).assign(
    positive_return = lambda x: (x['returns'] > 0) *1
))

IsADirectoryError: Path c:/Yuliia/DSI/scaling_to_production/05_src/data/prices/A/A_2000.parquet points to a directory, but only file paths are supported. To construct a nested or union dataset pass a list of dataset objects instead.

In [16]:
# Group by 'ticker' and apply transformations
dd_px = dd.read_parquet(parquet_files)
dd_feat = (
    dd_px.groupby('ticker', group_keys=False).apply(
        lambda x: x.assign(
            Close_lag=x['Close'].shift(1)
        ).assign(
            returns=lambda x: x['Close'] / x['Close_lag'] - 1
        ).assign(
            positive_return=lambda x: (x['returns'] > 0) * 1
        ),
        meta={
            'Close': 'f8', 'Close_lag': 'f8', 'returns': 'f8', 'positive_return': 'i4', 
            'ticker': 'O', 'High': 'f8', 'Low': 'f8', 'Adj Close': 'f8'
        }
    )
)

# Repartition to optimize performance (optional)
dd_feat = dd_feat.repartition(npartitions='auto')

# Compute and show results
dd_feat.head()


IsADirectoryError: Path c:/Yuliia/DSI/scaling_to_production/05_src/data/prices/A/A_2000.parquet points to a directory, but only file paths are supported. To construct a nested or union dataset pass a list of dataset objects instead.

+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [25]:
# Write your code below.



Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.